In [1]:
import pandas as pd
import numpy as np
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters
from tsfresh.feature_extraction import feature_calculators
from catboost import CatBoostClassifier, Pool, cv
from functools import partial
import multiprocessing as mp
from sklearn.model_selection import train_test_split, KFold
from tqdm.auto import tqdm
import librosa

In [2]:
train_data = pd.read_csv('/kaggle/input/tinkoff/train_gt.csv',header=None)
test_data = pd.read_csv('/kaggle/input/tinkoff/test-10.csv',header=None)
train_data.columns = ['path','label']
test_data.columns = ['path','label']

In [3]:
tqdm.pandas()
train_data['array'] = train_data['path'].progress_apply(lambda x: librosa.load(f'/kaggle/input/tinkoff/train-4/train/{x}')[0])
test_data['array'] = test_data['path'].progress_apply(lambda x: librosa.load(f'/kaggle/input/tinkoff/test-5/test/{x}')[0])

  0%|          | 0/8803 [00:00<?, ?it/s]

  0%|          | 0/2870 [00:00<?, ?it/s]

In [4]:
fc_parameters = ComprehensiveFCParameters()
pool = mp.Pool(mp.cpu_count())

for f_or_function_name, parameter_list in tqdm(fc_parameters.items()):
    func = getattr(feature_calculators, f_or_function_name)
    if parameter_list is not None:
        for params in parameter_list:
            func = partial(func,param=params)
            train_data[f'f_or_function_name_{params}'] = pool.map(func,train_data['array'])
            test_data[f'f_or_function_name_{params}'] = pool.map(func,test_data['array'])
            func = getattr(feature_calculators, f_or_function_name)
    else:   
        train_data[f_or_function_name] = pool.map(func,train_data['array'])
        test_data[f_or_function_name] = pool.map(func,test_data['array'])
    print('Iter')


  0%|          | 0/75 [00:00<?, ?it/s]

Iter


Iter


Iter


Iter


...

Iter


Iter


Iter


Iter


Iter


In [ ]:
train_data.to_csv('train_data.csv',index=False)
test_data.to_csv('test_data.csv',index=False)